# Catalogues
Here shown a possible implementation of the functionality useful to find the "main" column names of any catalogue, that is, the columns containing the main object_id, ra, and dec. 
I decided to load in memory (table _IRD) the main columns of all catalogs (such query is fast), and then filter the _IRD for the specific table_name provided by the user, every time the user asks for it.

In [1]:
from astroquery.eso import Eso
eso = Eso()

In [2]:
# Internal astropy table, loaded at start time, not exposed to users: _IRD (Id, Ra, Dec)
# containing the information of the column names carrying the main object Id, Ra, and Dec
# for each catalog.
# _IRD columns: table_name, column_name, ucd, unit
#
# Once the IRD is loaded in memory, main_cols function to return the main_cols dict for a table

#private
def _set_main_cols(table_name=None):
    """method that finds the principle columns id, ra, dec for a given table or for all tables"""
    constraint = 'where'
    if table_name:
        constraint = "where table_name = '%s' and" % (table_name)
    query = """SELECT table_name, column_name, ucd, unit
                from TAP_SCHEMA.columns
                %s
                (ucd = 'meta.id;meta.main' or 
                 ucd = 'pos.eq.ra;meta.main' or 
                 ucd = 'pos.eq.dec;meta.main')
                order by 1,3 desc""" % (constraint)
    return eso.query_tap(query, which_tap="tap_cat")

#private
_IRD = _set_main_cols()

In [3]:
#public
# should be part of the eso module and invoked as eso.main_cols
def main_cols(
    table_name
    ):
    """
    Find the columns names carrying the object id, ra, dec of a given catalogue.
    
    Parameters
    ----------
    table_name : str
        The table name of the catalogue.
    Returns
    -------
    dict
        Dictionary with the specific (id, ra, dec) column names used by the specific catalogue.
    """
    
    principal = {}
    # Not all tables have all 3 main cols:
    principal['ra'] = None
    principal['dec'] = None
    principal['id'] = None
    
    this = _IRD[ _IRD['table_name'] == table_name]

    match = this[ this['ucd'] == 'pos.eq.ra;meta.main' ]
    if len(match):
        principal['ra'] = match['column_name'][0]

    match = this[ this['ucd'] == 'pos.eq.dec;meta.main' ]
    if len(match):
        principal['dec'] = match['column_name'][0]

    match = this[ this['ucd'] == 'meta.id;meta.main' ]
    if len(match):
        principal['id'] = match['column_name'][0]

    return principal

#public
# should be part of the eso module and invoked as eso.make_cone_filter
def make_cone_filter(
       ra,
       dec,
       radius, 
        table_name
       ):
       """
       Construct an ADQL cone-search predicate for catalogue queries.

       Parameters
       ----------
       ra : astropy.units.Quantity
           Right ascension of cone centre.
       dec : astropy.units.Quantity
           Declination of cone centre.
       radius : astropy.units.Quantity
           Search radius.
       ra_name : str, optional
           Name of the RA column in the catalogue
           (default: 'RAJ2000').
       dec_name : str, optional
           Name of the Dec column in the catalogue
           (default: 'DECJ2000').

       Returns
       -------
       dict
           Dictionary suitable for ``column_filters`` of the form
           ``{predicate: 1}``, corresponding to
           ``CONTAINS(...) = 1`` in ADQL.
       """

       ra_deg = ra.to_value(u.deg)
       dec_deg = dec.to_value(u.deg)
       rad_deg = radius.to_value(u.deg)

       ird = main_cols(table_name)
    
       predicate = (
           f"CONTAINS("
           f"POINT('', {ird['ra']}, {ird['dec']}), "
           f"CIRCLE('', {ra_deg}, {dec_deg}, {rad_deg})"
           f")"
       )

       return {predicate: 1}

### Examples: how to get the main columns (object_id, ra, dec) of a catalog   

In [4]:
# Get the main columns of a PESSTO catalog
# I call the resulting dictionary ird to remember those are the Id Ra Dec column names
ird = main_cols('PESSTO_TRAN_CAT_fits_V2')   # should be eso.main_cols instead
print(ird['ra'], ird['dec'])
print(ird['id'])

TRANSIENT_RAJ2000 TRANSIENT_DECJ2000
TRANSIENT_ID


In [5]:
# Not all catalogs have all 3 main columns:
ird = main_cols('vmc_dr7_mPhotY_V6')   # should be eso.main_cols instead
print(ird['ra'], ird['dec'])
print(ird['id'])

None None
PHOT_ID


### Example of a cone search onto the KiDS catalog

In [6]:
# Defining a cone:
from astroquery.eso import Eso
from astropy.coordinates import SkyCoord
import astropy.units as u

coords = SkyCoord.from_name("NGC1097")
radius = 3 * u.arcmin

In [7]:
#Perparing a cone search constraint 
# via the make_cone_filter function, 

catalogue = "KiDS_DR4_1_ugriZYJHKs_cat_fits"
#catalogue = "PESSTO_TRAN_CAT_fits_V2"

column_filters = make_cone_filter(   # should be eso.make_cone_filter instead.
    ra=coords.ra,
    dec=coords.dec,
    radius=radius, table_name=catalogue
)

In [8]:
column_filters

{"CONTAINS(POINT('', RAJ2000, DECJ2000), CIRCLE('', 41.5794125, -30.27491111, 0.05))": 1}

In [9]:
table = eso.query_catalogue(
    catalogue="KiDS_DR4_1_ugriZYJHKs_cat_fits",
    column_filters=column_filters,
)
len(table) # expected 55

55